In [16]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

font_path = '/System/Library/AssetsV2/com_apple_MobileAsset_Font7/bad9b4bf17cf1669dde54184ba4431c22dcad27b.asset/AssetData/NanumGothic.ttc'
fontprop = fm.FontProperties(fname = font_path, size = 10)

from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.preprocessing import LabelEncoder

train = pd.read_csv('data/train.csv').drop(columns = 'ID')
test = pd.read_csv('data/test.csv')

In [3]:
train.head()

,매물확인방식,보증금,월세,전용면적,해당층,총층,방향,방수,욕실수,주차가능여부,총주차대수,관리비,중개사무소,제공플랫폼,게재일,허위매물여부
0,현장확인,402500000.0,470000,NaN,NaN,15.0,서향,1.0,1.0,가능,40.0,96,t93Nt6I2I0,B플랫폼,2024-10-09,0
1,현장확인,170500000.0,200000,NaN,3.0,4.0,남동향,2.0,1.0,불가능,NaN,0,q39iV5J4E6,D플랫폼,2024-12-26,0
2,전화확인,114000000.0,380000,NaN,2.0,3.0,동향,1.0,1.0,불가능,NaN,0,b03oE4G3F6,A플랫폼,2024-11-28,0
3,현장확인,163500000.0,30000,36.3,3.0,9.0,남동향,2.0,1.0,가능,13.0,10,G52Iz8V2B9,A플랫폼,2024-11-26,0
4,현장확인,346000000.0,530000,NaN,3.0,3.0,동향,2.0,1.0,불가능,NaN,0,N45gM0M7R0,B플랫폼,2024-06-25,1


# 기본적인 전처리 - 인코딩, 변수 생성 등

## 결측치 처리하기 전 인코딩부터 진행

In [4]:
if not pd.api.types.is_datetime64_any_dtype(train['게재일']):
    train['게재일'] = pd.to_datetime(train['게재일'], errors='coerce')  # 날짜 형식 변환

# 개제연 & 개제월 컬럼 생성
train['게재연'] = train['게재일'].dt.year
train['게재월'] = train['게재일'].dt.month

# 개제일 컬럼 삭제
train.drop(columns=['게재일'], inplace=True)

In [69]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

def encode_features(df, encoding_dict):
    """
    다양한 인코딩 방식(One-Hot, Label Encoding, Target Encoding)을 자동 처리하는 함수.
    결측치는 인코딩하지 않고 그대로 유지.

    Parameters:
    - df: DataFrame
    - encoding_dict: 각 컬럼과 인코딩 방식을 정의한 딕셔너리
        예: {"주차가능여부": "onehot", "플랫폼": "onehot", "중개사무소": "label", "다른컬럼": "target"}
    - target_column: Target Encoding에 사용할 타겟 컬럼 이름 (필요한 경우)

    Returns:
    - 인코딩된 DataFrame
    - LabelEncoder 객체를 저장한 딕셔너리
    """
    df_encoded = df.copy()
    label_encoders = {}  # LabelEncoder 객체 저장용 딕셔너리

    for column in encoding_dict.keys():
        if column in df_encoded.columns:  # 컬럼이 실제로 존재할 때만 처리
            encoding_type = encoding_dict[column]
            if encoding_type == "onehot":
                # 결측치를 포함한 원-핫 인코딩 수행
                df_encoded = pd.get_dummies(df_encoded, columns=[column], prefix=column, dummy_na=True)
            elif encoding_type == "label":
                # 결측치를 제외하고 Label Encoding 수행
                le = LabelEncoder()
                non_null_mask = df_encoded[column].notnull()
                df_encoded.loc[non_null_mask, column] = le.fit_transform(df_encoded.loc[non_null_mask, column].astype(str))
                label_encoders[column] = le
            elif encoding_type == "target":
                target_column = "허위매물여부"  # Target Encoding에 사용할 타겟 변수
                # Target Encoding: 각 범주를 타겟 변수의 평균값으로 치환
                encoding_map = df_encoded.groupby(column)[target_column].mean()
                df_encoded[column] = df_encoded[column].map(encoding_map)
            else:
                raise ValueError(f"Unsupported encoding type: {encoding_type}")
        else:
            print(f"Warning: Column '{column}' specified in encoding_dict is not in DataFrame columns.")
    bool_columns = df_encoded.select_dtypes(include='bool').columns
    df_encoded[bool_columns] = df_encoded[bool_columns].astype(int)

    # 필요 없는 열 삭제 (예: NaN 처리용 더미 열)
    if "주차가능여부_nan" in df_encoded.columns:
        df_encoded = df_encoded.drop(columns=['주차가능여부_nan'])

    return df_encoded


# 인코딩 방식 정의
encoding_dict = {
    "주차가능여부" : "onehot",
    "제공플랫폼" : "target",
    "매물확인방식" : "target",
    "중개사무소" : "target"  # Target Encoding 적용
}

train_encoded = encode_features(train, encoding_dict)

# bool_columns = train_encoded.select_dtypes(include='bool').columns
# train_encoded[bool_columns] = train_encoded[bool_columns].astype(int)

# 필요 없는 열 삭제 (예: NaN 처리용 더미 열)
# if "주차가능여부_nan" in train_encoded.columns:
#     train_encoded = train_encoded.drop(columns=['주차가능여부_nan'])

train_encoded.iloc[:,:15].head(3)

,매물확인방식,보증금,월세,전용면적,해당층,총층,방향,방수,욕실수,총주차대수,관리비,중개사무소,제공플랫폼,허위매물여부,게재연
0,0.131378,402500000.0,470000,NaN,NaN,15.0,서향,1.0,1.0,40.0,96,0.333333,0.104918,0,2024
1,0.131378,170500000.0,200000,NaN,3.0,4.0,남동향,2.0,1.0,NaN,0,0.000000,0.201087,0,2024
2,0.094241,114000000.0,380000,NaN,2.0,3.0,동향,1.0,1.0,NaN,0,0.052632,0.125261,0,2024


In [34]:
train_encoded.columns

Index(['매물확인방식', '보증금', '월세', '전용면적', '해당층', '총층', '방향', '방수', '욕실수', '총주차대수',
       '관리비', '중개사무소', '제공플랫폼', '허위매물여부', '게재연', '게재월', '주차가능여부_가능',
       '주차가능여부_불가능'],
      dtype='object')

## 월세 & 보증금 -> 단위 (만)으로 바꾸고 / 월세(만) + 관리비 컬럼(월세+관리비) 생성

In [35]:
# '보증금', '월세' (만) 단위로 변환
train_encoded['보증금'] = train_encoded['보증금'] / 10000
train_encoded['월세'] = train_encoded['월세'] / 10000

# 월세 + 관리비 새로운 컬럼 생성
train_encoded['월비합계'] = train_encoded['월세'] + train_encoded['관리비']
train_encoded = train_encoded.drop(columns=['월세', '관리비'])
train_encoded.head()

,매물확인방식,보증금,전용면적,해당층,총층,방향,방수,욕실수,총주차대수,중개사무소,제공플랫폼,허위매물여부,게재연,게재월,주차가능여부_가능,주차가능여부_불가능,월비합계
0,0.131378,40250.0,NaN,NaN,15.0,서향,1.0,1.0,40.0,0.333333,0.104918,0,2024,10,1,0,143.0
1,0.131378,17050.0,NaN,3.0,4.0,남동향,2.0,1.0,NaN,0.000000,0.201087,0,2024,12,0,1,20.0
2,0.094241,11400.0,NaN,2.0,3.0,동향,1.0,1.0,NaN,0.052632,0.125261,0,2024,11,0,1,38.0
3,0.131378,16350.0,36.3,3.0,9.0,남동향,2.0,1.0,13.0,0.000000,0.125261,0,2024,11,1,0,13.0
4,0.131378,34600.0,NaN,3.0,3.0,동향,2.0,1.0,NaN,0.052632,0.104918,1,2024,6,0,1,53.0


## 결측치 유무 컬럼 생성

In [36]:
# 결측치 유무 컬럼 생성
train_encoded['결측치_유무'] = train_encoded.isnull().any(axis=1).astype(int)
print("인코딩(라벨 & 원핫 & 타겟), 보증금 & 월세 단위 변경, 월세 + 관리비 생성 및 월세 & 관리비 제거, 결측치 유무 컬럼 생성 후 : ", train_encoded.shape)
print("위의 전처리 전 : ", train.shape)
print(train_encoded.columns)
train_encoded.head(3)

인코딩(라벨 & 원핫 & 타겟), 보증금 & 월세 단위 변경, 월세 + 관리비 생성 및 월세 & 관리비 제거, 결측치 유무 컬럼 생성 후 :  (2452, 18)
위의 전처리 전 :  (2452, 17)
Index(['매물확인방식', '보증금', '전용면적', '해당층', '총층', '방향', '방수', '욕실수', '총주차대수',
       '중개사무소', '제공플랫폼', '허위매물여부', '게재연', '게재월', '주차가능여부_가능', '주차가능여부_불가능',
       '월비합계', '결측치_유무'],
      dtype='object')


,매물확인방식,보증금,전용면적,해당층,총층,방향,방수,욕실수,총주차대수,중개사무소,제공플랫폼,허위매물여부,게재연,게재월,주차가능여부_가능,주차가능여부_불가능,월비합계,결측치_유무
0,0.131378,40250.0,NaN,NaN,15.0,서향,1.0,1.0,40.0,0.333333,0.104918,0,2024,10,1,0,143.0,1
1,0.131378,17050.0,NaN,3.0,4.0,남동향,2.0,1.0,NaN,0.000000,0.201087,0,2024,12,0,1,20.0,1
2,0.094241,11400.0,NaN,2.0,3.0,동향,1.0,1.0,NaN,0.052632,0.125261,0,2024,11,0,1,38.0,1


# 결측치 전처리

- '전용면적', '해당층', '총층', '방수', '욕실수', '총주차대수' 에 대한 전처리 필요
    - 전용 면적 : 클러스터링 or 코사인 유사도로 mean으로 fill
    - 해당층 & 총층 : drop 고려
    - 방수, 욕실수 : drop 고려
    - 총주차대수 : 주차유무만 살리면서 총주차대수는 drop 고려

### 전용면적 결측치 처리

- 전용면적 클러스터링 및 유사도 측정 피쳐 : 보증금, 방수, 욕실수, 월비합계, 주차가능여부_가능, 주차가능여부_불가능

In [37]:
missing_data_dict = {}
non_missing_data_dict = {}

for column in train.columns:
    missing_data = train[train[column].isnull()]
    if not missing_data.empty:
        missing_data_dict[column] = missing_data
        # print(f"\n[{column}]에 결측치가 있는 데이터 샘플:")
        # print(missing_data.head(1))

for column in train.columns:
    non_missing_data = train[train[column].notnull()]  # 결측치가 없는 데이터 선택
    if not non_missing_data.empty:
        non_missing_data_dict[column] = non_missing_data
        # print(f"\n[{column}]에 결측치가 없는 데이터 샘플:")
        # print(non_missing_data.head(1))

In [38]:
def fill_area(df, method, n_clusters=5, top_n=5):
    """
    전용면적의 결측치를 처리하는 함수.
    클러스터링(KMeans) 또는 코사인 유사도를 사용하여 결측치를 대체할 수 있음.

    Parameters:
    - method: 'clustering' 또는 'cosine'로 결측치 처리 방식 선택
        - n_clusters: 클러스터링 시 사용할 클러스터 수 (method='clustering'일 때 사용)  
        - top_n: 코사인 유사도 상위 N개의 평균값으로 결측치를 채움 (method='cosine'일 때 사용)  

    Returns:
    - 결측치가 처리된 DataFrame (df_filled)
    """
    df_filled = df.copy()

    area_non_missing_data = df_filled[df_filled['전용면적'].notnull()]
    area_missing_data = df_filled[df_filled['전용면적'].isnull()]

    # 피처 선택
    print("다음의 피처들 중에서 클러스터링 및 코사인 유사도 측정을 진행할 피처를 선택해주세요: \n", df_filled.columns)
    feature_columns = input("피처 열 이름을 쉼표로 구분하여 입력하세요: ").split(',')
    feature_columns = [col.strip() for col in feature_columns]
    print("\n선택된 피처:", feature_columns)

    if method == 'clustering':
        print("--------------- 전용 면적 전처리 시작 (클러스터링) ---------------")
        
        # 전용면적 결측치가 없는 데이터만 사용
        data_for_clustering = area_non_missing_data[feature_columns + ['전용면적']].copy()
        
        # 클러스터링
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        data_for_clustering['클러스터'] = kmeans.fit_predict(data_for_clustering[feature_columns])
        
        # 클러스터별 전용면적 평균 계산
        cluster_means = data_for_clustering.groupby('클러스터')['전용면적'].mean()

        # 결측치 데이터에 클러스터 할당
        def assign_cluster(row):
            feature_values = row[feature_columns].values.reshape(1, -1)
            return kmeans.predict(feature_values)[0]

        area_missing_data['클러스터'] = area_missing_data.apply(assign_cluster, axis=1)

        # 클러스터별 평균값으로 결측치 채움
        for cluster, mean_value in cluster_means.items():
            area_missing_data.loc[area_missing_data['클러스터'] == cluster, '전용면적'] = mean_value

        # 결측치 채운 데이터 합치기
        df_filled.update(area_missing_data)
        print("--------------- 전용 면적 전처리 완료 (클러스터링) ---------------")

    elif method == 'cosine':
        print("--------------- 전용 면적 전처리 시작 (코사인 유사도) ---------------")

        # 코사인 유사도 기반 결측치 처리
        for idx in area_missing_data.index:
            current_row = area_missing_data.loc[idx, feature_columns].values.reshape(1, -1)
            complete_features = area_non_missing_data[feature_columns].values

            similarities = cosine_similarity(current_row, complete_features).flatten()

            top_n_indices = np.argsort(similarities)[-top_n:]
            top_n_values = area_non_missing_data.iloc[top_n_indices]['전용면적']

            df_filled.at[idx, '전용면적'] = top_n_values.mean()

        print("--------------- 전용 면적 전처리 완료 (코사인 유사도) ---------------")

    else:
        raise ValueError("Invalid method. Use 'clustering' or 'cosine'.")
    
    return df_filled

df_area = fill_area(train_encoded, method = 'clustering', n_clusters=5)
print("결측치 처리 후 전용면적 결측치 수 : ",df_area['전용면적'].isna().sum())
df_area.head()

다음의 피처들 중에서 클러스터링 및 코사인 유사도 측정을 진행할 피처를 선택해주세요: 
 Index(['매물확인방식', '보증금', '전용면적', '해당층', '총층', '방향', '방수', '욕실수', '총주차대수',
       '중개사무소', '제공플랫폼', '허위매물여부', '게재연', '게재월', '주차가능여부_가능', '주차가능여부_불가능',
       '월비합계', '결측치_유무'],
      dtype='object')

선택된 피처: ['보증금', '주차가능여부_가능', '주차가능여부_불가능', '월비합계']
--------------- 전용 면적 전처리 시작 (클러스터링) ---------------
--------------- 전용 면적 전처리 완료 (클러스터링) ---------------
결측치 처리 후 전용면적 결측치 수 :  0


,매물확인방식,보증금,전용면적,해당층,총층,방향,방수,욕실수,총주차대수,중개사무소,제공플랫폼,허위매물여부,게재연,게재월,주차가능여부_가능,주차가능여부_불가능,월비합계,결측치_유무
0,0.131378,40250.0,26.933179,NaN,15.0,서향,1.0,1.0,40.0,0.333333,0.104918,0,2024,10,1,0,143.0,1
1,0.131378,17050.0,27.005774,3.0,4.0,남동향,2.0,1.0,NaN,0.000000,0.201087,0,2024,12,0,1,20.0,1
2,0.094241,11400.0,27.398170,2.0,3.0,동향,1.0,1.0,NaN,0.052632,0.125261,0,2024,11,0,1,38.0,1
3,0.131378,16350.0,36.300000,3.0,9.0,남동향,2.0,1.0,13.0,0.000000,0.125261,0,2024,11,1,0,13.0,0
4,0.131378,34600.0,26.933179,3.0,3.0,동향,2.0,1.0,NaN,0.052632,0.104918,1,2024,6,0,1,53.0,1


### 해당층 & 총층 전처리

In [39]:
na_features = ['총층', '해당층', '방수', '욕실수', '총주차대수']
for feature in na_features:
    # NaN 값이 있는 인덱스 추출 후 정렬
    na_indices = sorted(set(df_area[df_area[f'{feature}'].isna()].index))
    print(f"{feature}의 널데이터 매물 인덱스: {na_indices}")

총층의 널데이터 매물 인덱스: [37, 76, 88, 153, 339, 456, 515, 546, 686, 952, 1664, 1964, 2071, 2084, 2086, 2097]
해당층의 널데이터 매물 인덱스: [0, 5, 20, 28, 37, 61, 68, 72, 76, 78, 83, 88, 102, 106, 118, 125, 131, 133, 143, 153, 154, 175, 182, 190, 191, 197, 223, 229, 259, 280, 282, 300, 304, 339, 369, 376, 382, 405, 406, 414, 416, 418, 422, 427, 441, 444, 456, 465, 470, 480, 515, 522, 525, 536, 542, 544, 546, 554, 559, 573, 590, 650, 656, 660, 662, 663, 666, 669, 675, 686, 695, 707, 720, 726, 732, 742, 753, 785, 809, 822, 823, 845, 853, 857, 861, 862, 877, 891, 910, 916, 918, 920, 935, 947, 952, 957, 981, 1020, 1022, 1028, 1029, 1037, 1038, 1042, 1059, 1065, 1069, 1085, 1097, 1116, 1137, 1144, 1145, 1154, 1156, 1157, 1166, 1175, 1182, 1190, 1191, 1193, 1196, 1197, 1211, 1235, 1236, 1238, 1257, 1268, 1274, 1277, 1291, 1305, 1342, 1372, 1382, 1384, 1409, 1419, 1420, 1435, 1469, 1476, 1480, 1488, 1512, 1515, 1518, 1523, 1535, 1569, 1608, 1644, 1649, 1652, 1658, 1664, 1673, 1686, 1693, 1702, 1749, 1758, 1762, 1

In [40]:
# 총층, 해당층 drop
df_area_floor = df_area.drop(columns=['총층', '해당층'])
print("총층, 해당층 drop 전 결측치 :", df_area.isna().sum().sum())
print("총층, 해당층 drop 후 결측치 :", df_area_floor.isna().sum().sum())
df_area_floor.head(2)

총층, 해당층 drop 전 결측치 : 975
총층, 해당층 drop 후 결측치 : 730


,매물확인방식,보증금,전용면적,방향,방수,욕실수,총주차대수,중개사무소,제공플랫폼,허위매물여부,게재연,게재월,주차가능여부_가능,주차가능여부_불가능,월비합계,결측치_유무
0,0.131378,40250.0,26.933179,서향,1.0,1.0,40.0,0.333333,0.104918,0,2024,10,1,0,143.0,1
1,0.131378,17050.0,27.005774,남동향,2.0,1.0,NaN,0.000000,0.201087,0,2024,12,0,1,20.0,1


### 총주차대수 전처리

In [41]:
df_area_floor_parking = df_area_floor.drop(columns='총주차대수')
print("총주차대수 drop 전 결측치 :", df_area_floor.isna().sum().sum())
print("총주차대수 drop 후 결측치 :", df_area_floor_parking.isna().sum().sum())
df_area_floor_parking.head(2)

총주차대수 drop 전 결측치 : 730
총주차대수 drop 후 결측치 : 34


,매물확인방식,보증금,전용면적,방향,방수,욕실수,중개사무소,제공플랫폼,허위매물여부,게재연,게재월,주차가능여부_가능,주차가능여부_불가능,월비합계,결측치_유무
0,0.131378,40250.0,26.933179,서향,1.0,1.0,0.333333,0.104918,0,2024,10,1,0,143.0,1
1,0.131378,17050.0,27.005774,남동향,2.0,1.0,0.000000,0.201087,0,2024,12,0,1,20.0,1


### 방수 & 욕실수 전처리

In [42]:
print("방수 널데이터 수:",len(df_area_floor_parking[df_area_floor_parking['방수'].isna()]))
print("욕실수 널데이터 수:",len(df_area_floor_parking[df_area_floor_parking['욕실수'].isna()]))

방수 널데이터 수: 16
욕실수 널데이터 수: 18


In [43]:
df_area_floor_parking_room = df_area_floor_parking.drop(columns=['욕실수','방수'])
print("방수, 욕실수 drop 전 결측치 :", df_area_floor_parking.isna().sum().sum())
print("방수, 욕실수 drop 후 결측치 :", df_area_floor_parking_room.isna().sum().sum())
train_preprocessed_1 = df_area_floor_parking_room.copy()
train_preprocessed_1.head(2)

방수, 욕실수 drop 전 결측치 : 34
방수, 욕실수 drop 후 결측치 : 0


,매물확인방식,보증금,전용면적,방향,중개사무소,제공플랫폼,허위매물여부,게재연,게재월,주차가능여부_가능,주차가능여부_불가능,월비합계,결측치_유무
0,0.131378,40250.0,26.933179,서향,0.333333,0.104918,0,2024,10,1,0,143.0,1
1,0.131378,17050.0,27.005774,남동향,0.000000,0.201087,0,2024,12,0,1,20.0,1


# 피처 엔지니어링

## 변수 선택

- '보증금', '전용면적', '해당층', '총층', '방향', '방수', '욕실수', '총주차대수', '중개사무소',   
       '제공플랫폼', '허위매물여부', '게재연', '게재월', '매물확인방식_서류확인', '매물확인방식_전화확인',   
       '매물확인방식_현장확인', '주차가능여부_가능', '주차가능여부_불가능', '월비합계', '결측치_유무'

In [18]:
train_preprocessed_1.head()

,매물확인방식,보증금,전용면적,방향,중개사무소,제공플랫폼,허위매물여부,게재연,게재월,주차가능여부_가능,주차가능여부_불가능,월비합계,결측치_유무
0,2,40250.0,26.933179,7,240,1,0,2024,10,1,0,143.0,1
1,2,17050.0,27.005774,0,228,3,0,2024,12,0,1,20.0,1
2,1,11400.0,27.398170,3,150,0,0,2024,11,0,1,38.0,1
3,2,16350.0,36.300000,0,31,0,0,2024,11,1,0,13.0,0
4,2,34600.0,26.933179,3,72,1,1,2024,6,0,1,53.0,1


In [ ]:
train_preprocessed_1 = train_preprocessed_1.drop(columns='방향')
train_preprocessed_1.head()

KeyError: "['방향'] not found in axis"

In [72]:
print(train_preprocessed_1.columns)

Index(['매물확인방식', '보증금', '전용면적', '중개사무소', '제공플랫폼', '허위매물여부', '게재연', '게재월',
       '주차가능여부_가능', '주차가능여부_불가능', '월비합계', '결측치_유무'],
      dtype='object')


In [65]:
from sklearn.metrics import mean_squared_error
# 일부 데이터를 결측치로 변환하여 복원 성능 평가
simulated_missing_data = train_preprocessed_1.copy()
simulated_missing_data.loc[simulated_missing_data.sample(frac=0.1, random_state=42).index, '전용면적'] = np.nan

# 전처리 함수 적용 후 복원된 데이터와 원본 데이터 비교
restored_data = fill_area(simulated_missing_data, method='clustering', n_clusters=3, top_n = 3)
print("Restoration Performance (MSE):", mean_squared_error(train_preprocessed_1['전용면적'], restored_data['전용면적']))
# 보증금, 월비합계, 중개사무소, 제공플랫폼

다음의 피처들 중에서 클러스터링 및 코사인 유사도 측정을 진행할 피처를 선택해주세요: 
 Index(['매물확인방식', '보증금', '전용면적', '중개사무소', '제공플랫폼', '허위매물여부', '게재연', '게재월',
       '주차가능여부_가능', '주차가능여부_불가능', '월비합계', '결측치_유무'],
      dtype='object')

선택된 피처: ['월비합계', '중개사무소', '제공플랫폼']
--------------- 전용 면적 전처리 시작 (클러스터링) ---------------
--------------- 전용 면적 전처리 완료 (클러스터링) ---------------
Restoration Performance (MSE): 4.328533534980669


## 파생 변수 생성

# test 전처리

In [17]:
from preprocessor import datatime, encode_features, set_scale, make_null_column, fill_area, drop_null_data, select_features

test.head(2)

,ID,매물확인방식,보증금,월세,전용면적,해당층,총층,방향,방수,욕실수,주차가능여부,총주차대수,관리비,중개사무소,제공플랫폼,게재일
0,TEST_000,서류확인,223000000.0,530000,NaN,5.0,5.0,남향,1.0,1.0,불가능,NaN,7,Z86Th6S3K5,D플랫폼,2024-10-21
1,TEST_001,서류확인,150500000.0,590000,30.26,7.0,11.0,서향,1.0,1.0,불가능,16.0,11,G52Iz8V2B9,D플랫폼,2023-09-02


In [18]:
test_1 = datatime(test)
test_1.head(2)


 게재연 & 개제월 생성 및 개제일 제거


,ID,매물확인방식,보증금,월세,전용면적,해당층,총층,방향,방수,욕실수,주차가능여부,총주차대수,관리비,중개사무소,제공플랫폼,게재연,게재월
0,TEST_000,서류확인,223000000.0,530000,NaN,5.0,5.0,남향,1.0,1.0,불가능,NaN,7,Z86Th6S3K5,D플랫폼,2024,10
1,TEST_001,서류확인,150500000.0,590000,30.26,7.0,11.0,서향,1.0,1.0,불가능,16.0,11,G52Iz8V2B9,D플랫폼,2023,9


In [20]:
def calculate_target_encoding_map(train_df, column, target_column):
    """
    Train 데이터에서 Target Encoding 값을 계산.

    Parameters:
        train_df (pd.DataFrame): Train 데이터.
        column (str): 인코딩할 열 이름.
        target_column (str): 타겟 열 이름.

    Returns:
        dict: 범주와 타겟 평균값의 매핑 딕셔너리.
    """
    encoding_map = train_df.groupby(column)[target_column].mean().to_dict()
    return encoding_map

In [19]:
def apply_target_encoding(test_df, column, encoding_map, default_value):
    """
    Parameters:
        test_df (pd.DataFrame): Test 데이터.
        column (str): 인코딩할 열 이름.
        encoding_map (dict): Train 데이터에서 계산된 Target Encoding 매핑 딕셔너리.
        default_value (float): 매핑되지 않은 값에 사용할 기본값.
    """
    test_df[column] = test_df[column].map(encoding_map).fillna(default_value)
    return test_df

In [21]:
def encode_target_with_test(train_df, test_df, target_column, columns):
    """
    Train 데이터의 Target Encoding 값을 Test 데이터에 적용.

    Parameters:
        train_df (pd.DataFrame): Train 데이터.
        test_df (pd.DataFrame): Test 데이터.
        target_column (str): 타겟 열 이름.
        columns (list): Target Encoding을 적용할 열 리스트.

    Returns:
        tuple: (Train 데이터, Test 데이터)
    """
    for column in columns:
        # Train 데이터에서 Target Encoding 계산
        encoding_map = calculate_target_encoding_map(train_df, column, target_column)
        
        # Train 데이터에 Target Encoding 적용
        train_df[column] = train_df[column].map(encoding_map)
        
        # Test 데이터에 Target Encoding 적용
        global_mean = train_df[target_column].mean()  # 기본값: Train 타겟 평균값
        test_df = apply_target_encoding(test_df, column, encoding_map, default_value=global_mean)
    
    return train_df, test_df

In [ ]:
target_column = "허위매물여부"
target_encoding_columns = ["제공플랫폼", "매물확인방식", "중개사무소"]

# Train, Test 데이터 Target Encoding
train_data, test_data = encode_target_with_test(train_data, test_data, target_column, target_encoding_columns)

# 이후 Train 및 Test 데이터의 추가 전처리 실행
preprocessed_train_data = main(train_data)
preprocessed_test_data = preprocess_test_data(test_data, encoding_dict, scaler, cluster_centers)